In [1]:
import h5py
import numpy as np
from PIL import Image
import PIL
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import Sampler
from torch.utils.data.sampler import WeightedRandomSampler
import torchvision.datasets as datasetsgh
import torchvision.transforms as transforms
from torchsummary import summary

import rasterio
from rasterio.transform import Affine

import time
import copy
import sys
import random
import gc

In [42]:
PIL.Image.MAX_IMAGE_PIXELS = None

img_code = "QB02_20120729152314"
riley_path = f'data/data/riley_prediction/{img_code}_binary_10.tif'
model_path = f'data/data/maps_from_models/fracture_image_{img_code}.tif'

In [43]:
tile_side = 256
image_model = Image.open(model_path)
image_riley = Image.open(riley_path)

img_width, img_height = image_model.size
print(img_width, img_height)

cols = img_width // tile_side
rows = img_height // tile_side

fracture_density_model = Image.new('L', (img_width, img_height))
fracture_density_riley = Image.new('L', (img_width, img_height))

idx = 0
for i in range(rows):
    for j in range(cols):
        left = j * tile_side
        upper = i * tile_side
        right = left + tile_side
        lower = upper + tile_side
        
        model_tile = np.array(image_model.crop((left, upper, right, lower))) / 255
        riley_tile = np.array(image_riley.crop((left, upper, right, lower))) * 1
        
        #print(set(riley_tile.flatten().tolist()))
        FD_model = round(np.sum(model_tile) * 550 / (256 * 256))
        FD_riley = round(np.sum(riley_tile) * 550 / (256 * 256))
        
        model_tile_density = Image.fromarray(np.full((256, 256), FD_model, dtype=np.uint8))
        riley_tile_density = Image.fromarray(np.full((256, 256), FD_riley, dtype=np.uint8))
        
        x = j * 256
        y = i * 256
        
        fracture_density_model.paste(model_tile_density, (x, y))
        fracture_density_riley.paste(riley_tile_density, (x, y))
                                             

33024 33536


In [44]:
def create_tiff_from_image(image, source_tiff_path, output_tiff_path):
    # Open the source TIFF file to get metadata
    with rasterio.open(source_tiff_path) as src:
        metadata = src.meta.copy()
        transform = src.transform
        crs = src.crs
    
    array = np.array(image).astype(np.uint8)
    print(array.shape)

    metadata.update({
        'driver': 'GTiff',
        'height': array.shape[0],
        'width': array.shape[1],
        'count': 1,
        'dtype': array.dtype,
        'transform': transform,
        'crs': crs
    })

    compression_type = 'lzw'

    # Add the compression option to the metadata
    metadata.update(compress=compression_type)
    
    # Write the new TIFF file
    with rasterio.open(output_tiff_path, 'w', **metadata) as dst:
        dst.write(array, 1)  # Write the image data to band 1

In [45]:
PIL.Image.MAX_IMAGE_PIXELS = None

source_tiff_path = f"data/data/orig_tif_files/{img_code}.tif"
output_tiff_path = f"data/data/fracture_density_maps/model_{img_code}.tif"

create_tiff_from_image(fracture_density_model, source_tiff_path, output_tiff_path)

output_tiff_path = f"data/data/fracture_density_maps/riley_{img_code}.tif"

create_tiff_from_image(fracture_density_riley, source_tiff_path, output_tiff_path)

(33536, 33024)
(33536, 33024)
